In [8]:
# Profit Adjusted
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Load the Excel file
df = pd.read_excel('superhero_box_office.xlsx', parse_dates=['Release Date'])

# CPI data (approximate U.S. annual averages, base 2025 = 120.0)
cpi_data = {1989:46.5,1990:49.0,1991:51.1,1992:52.7,1993:54.2,1994:55.6,1995:57.2,1996:58.9,1997:60.2,1998:61.2,1999:62.5,2000:64.6,2001:66.4,
            2002:67.5,2003:69.0,2004:70.9,2005:73.3,2006:75.6,2007:77.8,2008:80.8,2009:80.5,2010:81.8,2011:84.4,2012:86.1,2013:87.4,2014:88.8,
            2015:88.9,2016:90.0,2017:92.0,2018:94.2,2019:95.9,2020:97.1,2021:101.7,2022:109.8,2023:114.4,2024:117.7,2025:120.0}

# Function to get CPI for a release year
def get_cpi(year):
    return cpi_data.get(year, cpi_data[2025])

# Adjust grosses for inflation (to 2025 dollars, in millions)
cpi_2025 = cpi_data[2025]
df['Year'] = df['Release Date'].dt.year
df['Release Date'] = df['Release Date']
df['Adjusted Opening Weekend (Millions)'] = (df['Opening Weekend'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Profit (Millions)'] = ((df['Worldwide Revenue'] - df['Total Budget']) * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Opening Weekend Percentage of Total Gross'] = df['Percent of Total Gross']
df['Domestic Box Office Share'] = df['Domestic Share Percentage']
df['International Box Office Share'] = (100 - df['Domestic Share Percentage'])

# Sort by Release Date
df = df.sort_values('Release Date')

# Create the figure
fig = go.Figure()

# Unique franchises and series
franchises = ['MCU', 'DC', 'FOX', 'SONY']
unique_series = {franchise: sorted(df[df['Franchise'] == franchise]['Series'].unique()) for franchise in franchises}

# Styling
colors = {
    'MCU': ('red', 'red', 'orange', 'cyan', 'magenta'),
    'DC': ('blue', 'blue', 'orange', 'cyan', 'magenta'),
    'SONY': ('black', 'black', 'orange', 'cyan', 'magenta'),
    'FOX': ('yellow', 'yellow', 'orange', 'cyan', 'magenta')
}
markers = {'MCU': 'circle', 'DC': 'square', 'SONY': 'triangle-up', 'FOX': 'diamond'}

# Add franchise traces
franchise_traces = {}
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_pro, color_open, color_owp, color_dom, color_int = colors.get(franchise, ('grey', 'darkgrey', 'blue', 'cyan', 'magenta'))
    marker = markers.get(franchise, 'circle')

    franchise_traces[franchise] = []
    
    # Profit
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Profit (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Profit',
        line=dict(color=color_pro, dash='solid'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Profit (Millions)'].round(2).astype(str) + 'M',
        visible=True))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    
    # Opening Weekend
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Opening Weekend (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Opening Weekend',
        line=dict(color=color_open, dash='dot'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Opening Weekend (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Opening Weekend Percentage
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Opening Weekend Percentage of Total Gross'],
        mode='lines+markers',
        name=f'{franchise} Opening Weekend % of Total',
        line=dict(color=color_owp, dash='dash'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Opening Weekend Percentage of Total Gross'].round(2).astype(str) + '%',
        yaxis='y2',
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Domestic Box Office Share (Bar)
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['Domestic Box Office Share'],
        name=f'{franchise} Domestic Share %',
        marker=dict(color=color_dom),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Domestic Box Office Share'].round(2).astype(str) + '%',
        yaxis='y2',
        opacity=0.3,
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # International Box Office Share (Bar)
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['International Box Office Share'],
        name=f'{franchise} International Share %',
        marker=dict(color=color_int),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['International Box Office Share'].round(2).astype(str) + '%',
        yaxis='y2',
        opacity=0.3,
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)

# Initialize Series traces
series_traces = {franchise: {} for franchise in franchises}
series_traces_start_index = len(fig.data)

# Add Series traces
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_pro, color_open, color_owp, color_dom, color_int = colors.get(franchise, ('grey', 'darkgrey', 'blue', 'cyan', 'magenta'))
    marker = markers.get(franchise, 'circle')
    
    for series in unique_series[franchise]:
        df_series = df_franchise[df_franchise['Series'] == series]
        if not df_series.empty:
            series_traces[franchise][series] = []
            
            # Series Total Revenue
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Profit (Millions)'],
                mode='lines+markers',
                name=f'{series} Profit or Loss',
                line=dict(color=color_pro, dash='solid'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Profit (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            
            # Series Opening Weekend
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Opening Weekend (Millions)'],
                mode='lines+markers',
                name=f'{series} Opening Weekend',
                line=dict(color=color_open, dash='dot'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Opening Weekend (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Opening Weekend Percentage
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Opening Weekend Percentage of Total Gross'],
                mode='lines+markers',
                name=f'{series} Opening Weekend % of Total',
                line=dict(color=color_owp, dash='dash'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Opening Weekend Percentage of Total Gross'].round(2).astype(str) + '%',
                yaxis='y2',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Domestic Box Office Share (Bar)
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['Domestic Box Office Share'],
                name=f'{series} Domestic Share',
                marker=dict(color=color_dom),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Domestic Box Office Share'].round(2).astype(str) + '%',
                yaxis='y2',
                opacity=0.3,
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series International Box Office Share (Bar)
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['International Box Office Share'],
                name=f'{series} International Share',
                marker=dict(color=color_int),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['International Box Office Share'].round(2).astype(str) + '%',
                yaxis='y2',
                opacity=0.3,
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)

# Create franchise dropdown
franchise_dropdown = [
    # ALL: Show all franchise traces
    dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in [0, 5, 10, 15] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)},
        {'updatemenus[1].buttons': 
        [dict(label='None',
        method='update',
        args=[{'visible': [True if i in [0, 5, 10, 15] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)}])],
        'updatemenus[1].visible': False  # Hide series dropdown when ALL selected
        }]
        ),
    # MCU
    dict(label='MCU',
         method='update',
         args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['MCU'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['MCU']],
                'updatemenus[1].visible': True}]
         ),
    # DC
    dict(label='DC',
         method='update',
         args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['DC'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['DC']],
                'updatemenus[1].visible': True}]
         ),
    # FOX
    dict(label='FOX',
         method='update',
         args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['FOX'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['FOX']],
                'updatemenus[1].visible': True}]
         ),
    # SONY
    dict(label='SONY',
         method='update',
         args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['SONY'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['SONY']],
                'updatemenus[1].visible': True}]
         )
]

# Default series dropdown (empty initially)
series_dropdown = [
    dict(label='None',
         method='update',
         args=[{'visible': [True] * len(franchises) * 5 + [False] * (len(fig.data) - len(franchises) * 5)}])
]

# Update layout
fig.update_layout(
    title=dict(text="Estimated Superhero Movie Profit (Inflation-Adjusted 2025)", font=dict(size=40), font_family='Garamond', automargin=True, yref='container', x=0.5, xanchor='center', y=.98, yanchor='top'),
    xaxis_title='Release Date',
    yaxis=dict(title='Adjusted Amount (Millions USD)', side='left'),
    yaxis2=dict(title='Percentage', side='right', overlaying='y', range=[0, 100], tickformat='.0f'),
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    barmode='stack',
    updatemenus=[
        dict(
            buttons=franchise_dropdown,
            direction='down',
            showactive=True,
            x=0.15,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0
        ),
        dict(
            buttons=series_dropdown,
            direction='down',
            showactive=True,
            x=0.2,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0,
            visible=False
        )
    ],
    xaxis=dict(range=[df['Release Date'].min() - pd.Timedelta(days=365), df['Release Date'].max() + pd.Timedelta(days=365)])
)

# Show the figure in Jupyter
#fig.show()

# Save as HTML
fig.write_html('EstimatedSuperheroProfit.html')# Profit Adjusted


In [1]:
#Profit
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Load the Excel file
df = pd.read_excel('superhero_box_office.xlsx', parse_dates=['Release Date'])

# CPI data (U.S. annual averages, base 2025 = 120.0)
cpi_data = {1989:46.5,1990:49.0,1991:51.1,1992:52.7,1993:54.2,1994:55.6,1995:57.2,1996:58.9,1997:60.2,1998:61.2,1999:62.5,2000:64.6,2001:66.4,
            2002:67.5,2003:69.0,2004:70.9,2005:73.3,2006:75.6,2007:77.8,2008:80.8,2009:80.5,2010:81.8,2011:84.4,2012:86.1,2013:87.4,2014:88.8,
            2015:88.9,2016:90.0,2017:92.0,2018:94.2,2019:95.9,2020:97.1,2021:101.7,2022:109.8,2023:114.4,2024:117.7,2025:120.0}

# Function to get CPI for a release year
def get_cpi(year):
    return cpi_data.get(year, cpi_data[2025])

# Adjust grosses for inflation (to 2025 dollars, in millions)
cpi_2025 = cpi_data[2025]
df['Year'] = df['Release Date'].dt.year
df['Release Date'] = df['Release Date']
df['Adjusted Opening Weekend (Millions)'] = (df['Opening Weekend'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Profit (Millions)'] = ((df['Worldwide Revenue'] - df['Total Budget']) * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Domestic Box Office Share'] = df['Domestic Share Percentage']
df['International Box Office Share'] = (100 - df['Domestic Share Percentage'])

# Sort by Release Date
df = df.sort_values('Release Date')

# Create the figure
fig = go.Figure()

# Unique franchises and series
franchises = ['MCU', 'DC', 'FOX', 'SONY']
unique_series = {franchise: sorted(df[df['Franchise'] == franchise]['Series'].unique()) for franchise in franchises}

# Styling
colors = {
    'MCU': ('red', 'orange', 'cyan', 'magenta'),
    'DC': ('blue', 'orange', 'cyan', 'magenta'),
    'SONY': ('black', 'orange', 'cyan', 'magenta'),
    'FOX': ('yellow', 'orange', 'cyan', 'magenta')
}
markers = {'MCU': 'circle', 'DC': 'square', 'SONY': 'triangle-up', 'FOX': 'diamond'}

# Add franchise traces
franchise_traces = {}
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_pro, color_open, color_dom, color_int = colors.get(franchise, ('grey', 'darkgrey', 'cyan', 'magenta'))
    marker = markers.get(franchise, 'circle')

    franchise_traces[franchise] = []
    
    # Profit
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Profit (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Profit',
        line=dict(color=color_pro, dash='solid'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Profit (Millions)'].round(2).astype(str) + 'M',
        visible=True))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Opening Weekend
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Opening Weekend (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Opening Weekend',
        line=dict(color=color_open, dash='dot'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Opening Weekend (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Domestic Box Office Share (Bar)
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['Domestic Box Office Share'],
        name=f'{franchise} Domestic Share %',
        marker=dict(color=color_dom),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Domestic Box Office Share'].round(2).astype(str) + '%',
        yaxis='y2',
        opacity=0.3,
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # International Box Office Share (Bar)
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['International Box Office Share'],
        name=f'{franchise} International Share %',
        marker=dict(color=color_int),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['International Box Office Share'].round(2).astype(str) + '%',
        yaxis='y2',
        opacity=0.3,
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)

# Add series traces
series_traces = {franchise: {} for franchise in franchises}
series_traces_start_index = len(fig.data)

for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_pro, color_open, color_dom, color_int = colors.get(franchise, ('grey', 'darkgrey', 'cyan', 'magenta'))
    marker = markers.get(franchise, 'circle')
    
    for series in unique_series[franchise]:
        df_series = df_franchise[df_franchise['Series'] == series]
        if not df_series.empty:
            series_traces[franchise][series] = []
            
            # Series Profit
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Profit (Millions)'],
                mode='lines+markers',
                name=f'{series} Profit',
                line=dict(color=color_pro, dash='solid'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Profit (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Opening Weekend
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Opening Weekend (Millions)'],
                mode='lines+markers',
                name=f'{series} Opening Weekend',
                line=dict(color=color_open, dash='dot'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Opening Weekend (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Domestic Box Office Share (Bar)
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['Domestic Box Office Share'],
                name=f'{series} Domestic Share',
                marker=dict(color=color_dom),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Domestic Box Office Share'].round(2).astype(str) + '%',
                yaxis='y2',
                opacity=0.3,
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series International Box Office Share (Bar)
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['International Box Office Share'],
                name=f'{series} International Share',
                marker=dict(color=color_int),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['International Box Office Share'].round(2).astype(str) + '%',
                yaxis='y2',
                opacity=0.3,
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)

# Create franchise dropdown
franchise_dropdown = [
    # ALL: Show only Profit franchise traces
    dict(label='ALL',
         method='update',
         args=[{'visible': [True if i in [0, 4, 8, 12] else False for i in range(len(franchises) * 4)] + [False] * (len(fig.data) - len(franchises) * 4)},
               {'updatemenus[1].buttons': 
                [dict(label='None',
                      method='update',
                      args=[{'visible': [True if i in [0, 4, 8, 12] else False for i in range(len(franchises) * 4)] + [False] * (len(fig.data) - len(franchises) * 4)}])],
                'updatemenus[1].visible': False  # Hide series dropdown when ALL selected
               }]
         ),
    # MCU
    dict(label='MCU',
         method='update',
         args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['MCU'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['MCU']],
                'updatemenus[1].visible': True}]
         ),
    # DC
    dict(label='DC',
         method='update',
         args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['DC'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['DC']],
                'updatemenus[1].visible': True}]
         ),
    # FOX
    dict(label='FOX',
         method='update',
         args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['FOX'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['FOX']],
                'updatemenus[1].visible': True}]
         ),
    # SONY
    dict(label='SONY',
         method='update',
         args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['SONY'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['SONY']],
                'updatemenus[1].visible': True}]
         )
]

# Default series dropdown (empty initially)
series_dropdown = [
    dict(label='None',
         method='update',
         args=[{'visible': [True if i in [0, 4, 8, 12] else False for i in range(len(franchises) * 4)] + [False] * (len(fig.data) - len(franchises) * 4)}])
]

# Update layout
fig.update_layout(
    title=dict(text="Estimated Superhero Movie Profits (Inflation-Adjusted 2025)", font=dict(size=40), font_family='Garamond', automargin=True, yref='container', x=0.5, xanchor='center', y=0.98, yanchor='top'),
    xaxis_title='Release Date',
    yaxis=dict(title='Adjusted Amount (Millions USD)', side='left'),
    yaxis2=dict(title='Percentage', side='right', overlaying='y', range=[0, 100], tickformat='.0f'),
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    barmode='stack',
    updatemenus=[
        dict(
            buttons=franchise_dropdown,
            direction='down',
            showactive=True,
            x=0.15,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0
        ),
        dict(
            buttons=series_dropdown,
            direction='down',
            showactive=True,
            x=0.2,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0,
            visible=False
        )
    ],
    xaxis=dict(range=[df['Release Date'].min() - pd.Timedelta(days=365), df['Release Date'].max() + pd.Timedelta(days=365)])
)

# Show the figure in Jupyter
#fig.show()

# Save as HTML
fig.write_html('EstimatedSuperheroMovieProfit.html')